In [3]:
import sqlite3
import pandas as pd

In [4]:
conn = sqlite3.connect('C:\\Users\\lahat\\Downloads\\FPA_FOD_20170508.sqlite')
df = pd.read_sql_query("SELECT * from Fires", conn)

In [5]:
df.head()

,OBJECTID,FOD_ID,FPA_ID,SOURCE_SYSTEM_TYPE,SOURCE_SYSTEM,NWCG_REPORTING_AGENCY,NWCG_REPORTING_UNIT_ID,NWCG_REPORTING_UNIT_NAME,SOURCE_REPORTING_UNIT,SOURCE_REPORTING_UNIT_NAME,...,FIRE_SIZE_CLASS,LATITUDE,LONGITUDE,OWNER_CODE,OWNER_DESCR,STATE,COUNTY,FIPS_CODE,FIPS_NAME,Shape
0,463445,497959,SFO-ID0043_3101998026,NONFED,ST-NASF,ST/C&L,USIDIDS,Idaho Department of Lands,ID310,IDS District 310,...,B,47.600845,-116.306789,14.0,MISSING/NOT SPECIFIED,ID,Kootenai,055,Kootenai,b'\x00\x01\xad\x10\x00\x00pqTn\xa2\x13]\xc0\x9...
1,1464479,201104109,W-626311,FED,DOI-WFMI,BIA,USMTFBA,Fort Belknap Agency,MTFBA,Fort Belknap Agency,...,A,48.403300,-108.289600,8.0,PRIVATE,MT,None,None,None,b'\x00\x01\xad\x10\x00\x00\xf8:p\xce\x88\x12[\...
2,1591094,201608816,SFO-NE-2012-16029,NONFED,ST-NASF,ST/C&L,USNENES,Nebraska Department of Forestry,NENFS,Nebraska Forest Service,...,B,40.668460,-99.089130,14.0,MISSING/NOT SPECIFIED,NE,None,None,None,b'\x00\x01\xad\x10\x00\x00\xe8\xc5PN\xb4\xc5X\...
3,944726,1072277,SWRA_VA_11300,NONFED,ST-VAVAS,ST/C&L,USVAVAS,Virginia Department of Forestry,VAVAS3,Farmville District,...,B,39.000000,-78.269200,14.0,MISSING/NOT SPECIFIED,VA,None,None,None,b'\x00\x01\xad\x10\x00\x00P\x05\xa3\x92:\x91S\...
4,896277,1022153,SWRA_LA_24392,NONFED,ST-LALAS,ST/C&L,USLALAS,Louisiana Office of Forestry,LALAS1,LAS District 1,...,B,30.993300,-89.863900,14.0,MISSING/NOT SPECIFIED,LA,None,None,None,b'\x00\x01\xad\x10\x00\x00\xe8\xc09#JwV\xc0\xf...


In [6]:
df.columns

Index(['OBJECTID', 'FOD_ID', 'FPA_ID', 'SOURCE_SYSTEM_TYPE', 'SOURCE_SYSTEM',
       'NWCG_REPORTING_AGENCY', 'NWCG_REPORTING_UNIT_ID',
       'NWCG_REPORTING_UNIT_NAME', 'SOURCE_REPORTING_UNIT',
       'SOURCE_REPORTING_UNIT_NAME', 'LOCAL_FIRE_REPORT_ID',
       'LOCAL_INCIDENT_ID', 'FIRE_CODE', 'FIRE_NAME',
       'ICS_209_INCIDENT_NUMBER', 'ICS_209_NAME', 'MTBS_ID', 'MTBS_FIRE_NAME',
       'COMPLEX_NAME', 'FIRE_YEAR', 'DISCOVERY_DATE', 'DISCOVERY_DOY',
       'DISCOVERY_TIME', 'STAT_CAUSE_CODE', 'STAT_CAUSE_DESCR', 'CONT_DATE',
       'CONT_DOY', 'CONT_TIME', 'FIRE_SIZE', 'FIRE_SIZE_CLASS', 'LATITUDE',
       'LONGITUDE', 'OWNER_CODE', 'OWNER_DESCR', 'STATE', 'COUNTY',
       'FIPS_CODE', 'FIPS_NAME', 'Shape'],
      dtype='object')

In [7]:
df[['STAT_CAUSE_CODE', 'STAT_CAUSE_DESCR']].drop_duplicates()

,STAT_CAUSE_CODE,STAT_CAUSE_DESCR
0,13.0,Missing/Undefined
1,9.0,Miscellaneous
4,7.0,Arson
5,5.0,Debris Burning
6,3.0,Smoking
8,4.0,Campfire
12,2.0,Equipment Use
17,11.0,Powerline
22,1.0,Lightning
33,6.0,Railroad


In [8]:
df.iloc[:, 26:]

,CONT_DOY,CONT_TIME,FIRE_SIZE,FIRE_SIZE_CLASS,LATITUDE,LONGITUDE,OWNER_CODE,OWNER_DESCR,STATE,COUNTY,FIPS_CODE,FIPS_NAME,Shape
0,NaN,None,2.00,B,47.600845,-116.306789,14.0,MISSING/NOT SPECIFIED,ID,Kootenai,055,Kootenai,b'\x00\x01\xad\x10\x00\x00pqTn\xa2\x13]\xc0\x9...
1,234.0,1800,0.10,A,48.403300,-108.289600,8.0,PRIVATE,MT,None,None,None,b'\x00\x01\xad\x10\x00\x00\xf8:p\xce\x88\x12[\...
2,140.0,1630,1.50,B,40.668460,-99.089130,14.0,MISSING/NOT SPECIFIED,NE,None,None,None,b'\x00\x01\xad\x10\x00\x00\xe8\xc5PN\xb4\xc5X\...
3,NaN,None,2.00,B,39.000000,-78.269200,14.0,MISSING/NOT SPECIFIED,VA,None,None,None,b'\x00\x01\xad\x10\x00\x00P\x05\xa3\x92:\x91S\...
4,NaN,None,5.00,B,30.993300,-89.863900,14.0,MISSING/NOT SPECIFIED,LA,None,None,None,b'\x00\x01\xad\x10\x00\x00\xe8\xc09#JwV\xc0\xf...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
626817,149.0,2300,0.28,B,43.032319,-73.403029,14.0,MISSING/NOT SPECIFIED,NY,WASHINGTON,115,Washington,b'\x00\x01\xad\x10\x00\x00\xa8.};\xcbYR\xc0\x8...
626818,194.0,1223,0.01,A,31.052800,-83.297100,8.0,PRIVATE,GA,Cook,075,Cook,b'\x00\x01\xad\x10\x00\x00\x0c\xe9\xb7\xaf\x03...
626819,123.0,1235,0.80,B,33.984400,-85.003100,14.0,MISSING/NOT SPECIFIED,GA,Polk,233,Polk,b'\x00\x01\xad\x10\x00\x00\x84\xa7W\xca2@U\xc0...
626820,NaN,None,0.30,B,37.048889,-121.981944,14.0,MISSING/NOT SPECIFIED,CA,None,None,None,b'\x00\x01\xad\x10\x00\x00|\x12~-\xd8~^\xc0\x8...
